If you need to install sentinelsat

In [ ]:
!pip install sentinelsat

In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import os
import geopandas as gpd
import zipfile
import getpass

In [2]:
user = 'bludka'
pword = getpass.getpass('Earthdata Login password: ')

Earthdata Login password: ········


In [3]:
#login into api
api = SentinelAPI(user, pword, 'https://scihub.copernicus.eu/dhus')

In [4]:
start_date = '20190101'
end_date = '20200331'

In [5]:
reef_footprint = geojson_to_wkt(read_geojson('Moce.geojson'))

In [6]:
#query sentinel sat api
products = api.query(reef_footprint,date = (start_date, end_date),platformname = 'Sentinel-2'\
                     ,area_relation = 'Intersects',processinglevel = 'Level-2A',\
                     order_by = 'cloudcoverpercentage')


In [7]:
# convert to Pandas DataFrame
products_df = api.to_dataframe(products)
products_df.head().T

,abae5ee5-946e-4028-8711-ba8c97bde611,456600f2-ce58-4e14-90df-51f980272ea5,58b60422-8883-4715-8951-9dbe6af3367c,207cb9ee-4cc3-469d-aee3-91980da8c3c4,aa993830-21ca-48f6-9dce-292947712fcf
title,S2B_MSIL2A_20190605T220929_N0212_R129_T01KCV_2...,S2A_MSIL2A_20190809T220921_N0213_R129_T01KCV_2...,S2B_MSIL2A_20190715T220929_N0213_R129_T01KCV_2...,S2B_MSIL2A_20200121T220909_N0213_R129_T01KCV_2...,S2A_MSIL2A_20191127T220921_N0213_R129_T01KCV_2...
link,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
link_alternative,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
link_icon,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
summary,"Date: 2019-06-05T22:09:29.024Z, Instrument: MS...","Date: 2019-08-09T22:09:21.024Z, Instrument: MS...","Date: 2019-07-15T22:09:29.024Z, Instrument: MS...","Date: 2020-01-21T22:09:09.024Z, Instrument: MS...","Date: 2019-11-27T22:09:21.024Z, Instrument: MS..."
beginposition,2019-06-05 22:09:29.024000,2019-08-09 22:09:21.024000,2019-07-15 22:09:29.024000,2020-01-21 22:09:09.024000,2019-11-27 22:09:21.024000
endposition,2019-06-05 22:09:29.024000,2019-08-09 22:09:21.024000,2019-07-15 22:09:29.024000,2020-01-21 22:09:09.024000,2019-11-27 22:09:21.024000
ingestiondate,2019-06-06 03:09:01.984000,2019-08-10 03:27:46.358000,2019-07-16 05:17:31.293000,2020-01-22 01:31:08.397000,2019-11-28 03:09:58.524000
orbitnumber,11738,21576,12310,15027,23149
relativeorbitnumber,129,129,129,129,129


In [8]:
#creating folder for saving sentinel images
sentinel_path = 'SAFE files'
if not os.path.exists(sentinel_path):
    os.makedirs(sentinel_path)

In [9]:
#downloading num_images
num_images = 1
for i,x in enumerate(products.items()):
    k,v = x[0],x[1]
    if i < num_images:
        api.download(k, directory_path = sentinel_path)

Downloading: 100%|██████████| 775M/775M [21:06<00:00, 612kB/s]   
MD5 checksumming: 100%|██████████| 775M/775M [00:01<00:00, 480MB/s] 


In [10]:
#unzipping files
for file in os.listdir(sentinel_path):
    if file.endswith('.zip'):
        file_path = os.path.join(sentinel_path, file)
        out_path = os.path.join(sentinel_path, file.split('.')[0])

        if os.path.exists(file_path) and not os.path.exists(out_path):
            with zipfile.ZipFile(file_path,"r") as zip_ref:
                zip_ref.extractall(sentinel_path)
            os.remove(file_path)